In [ ]:
# Code by MUHAMMAD ASHBIL SHAHID & MUHAMMAD SALMAN
# IMPORTANT: RUN THIS CELL IN ORDER
# FEEL FREE TO CHECK IT OUT ON GITHUB: https://github.com/AshbilShahid/Enhanced-Social-Media-Sentiment-Analysis
# THANK YOU FOR YOUR TIME!

# Necessary Libraries

In [ ]:
!pip install gradio
import joblib
import gradio as gr
import numpy as np
import re
from textblob import TextBlob
import plotly.express as px
import plotly.graph_objects as go

# Upload Model and Vectorizer

In [ ]:
!wget https://github.com/AshbilShahid/Enhanced-Social-Media-Sentiment-Analysis/raw/main/sentiment_model.pkl
!wget https://github.com/AshbilShahid/Enhanced-Social-Media-Sentiment-Analysis/raw/main/vectorizer.pkl
# Load the trained model and vectorizer
try:
    model = joblib.load('/content/sentiment_model.pkl')
    vectorizer = joblib.load('/content/vectorizer.pkl')
    print("Model and vectorizer loaded successfully.")
except Exception as e:
    print(f"Error loading model or vectorizer: {e}")

# Load the Necessary Functions

In [7]:
# Text Preprocessing Function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Prediction Function with Enhanced Feedback
def predict_sentiment(text):
    try:
        # Preprocess the input text
        clean_text = preprocess_text(text)

        # Vectorize and predict
        text_tfidf = vectorizer.transform([clean_text])
        prediction = model.predict(text_tfidf)[0]
        prediction_proba = model.predict_proba(text_tfidf)[0]
        confidence_score = round(np.max(prediction_proba) * 100, 2)  # Confidence in %

        # Effectiveness Calculation using TextBlob
        polarity = TextBlob(clean_text).sentiment.polarity
        effectiveness_score = max(0, min(100, confidence_score + (polarity * 20)))

        # Improvement Suggestions
        if effectiveness_score < 60:
            improvement_suggestions = "Consider clearer, more vivid language to convey sentiment."
        elif prediction == "positive":
            improvement_suggestions = "Add emotional depth for better engagement."
        elif prediction == "negative":
            improvement_suggestions = "Add specific examples or context to support your point."
        else:
            improvement_suggestions = "Try focusing on clarity for a stronger message."

        # Plotly Confidence Plot
        labels = ["Negative", "Neutral", "Positive"]
        fig = go.Figure(data=[go.Bar(x=labels, y=prediction_proba, marker_color=['red', 'gray', 'green'])])
        fig.update_layout(
            title="Sentiment Confidence Scores",
            xaxis_title="Sentiment",
            yaxis_title="Confidence",
            yaxis=dict(range=[0, 1])
        )

        # Return the structured output for Gradio
        return prediction, f"{confidence_score}%", f"{effectiveness_score}%", improvement_suggestions, fig

    except Exception as e:
        print(f"Error in prediction function: {e}")
        return "Error", "Error", "Error", "Error", go.Figure()  # Return empty figure on error

# Gradio Interface

In [ ]:
# Be sure to Click the gradio Link after the Function of the Cell Complete to Launch the Interface
# Set up the Gradio interface with proper outputs
interface = gr.Interface(
    fn=predict_sentiment,
    inputs="text",
    outputs=[
        gr.Textbox(label="Sentiment Prediction"),
        gr.Textbox(label="Confidence Score (%)"),
        gr.Textbox(label="Effectiveness Score (%)"),
        gr.Textbox(label="Improvement Suggestions"),
        gr.Plot(label="Confidence Plot")
    ],
    title="Enhanced Social Media Sentiment Analysis",
    description="Enter text to analyze its sentiment, confidence, effectiveness, and get suggestions for improvement.",
    examples=["I love this product!", "This is the worst experience I've ever had.", "I'm not sure about this..."]
)

# Launch the Gradio interface
interface.launch()